In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Dataset

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.19.0


# Inicjacja przestrzeni roboczej

In [2]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

# Utworzenie eksperymentu Azure ML

In [ ]:
from azureml.core import Experiment

script_folder = './scripts'
os.makedirs(script_folder, exist_ok=True)

retrain_folder = './scripts/retrain'
os.makedirs(retrain_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='textgen')

In [ ]:

dataset = Dataset.get_by_name(ws, name='korpus')

mount_ctx = dataset.mount()  
mount_ctx.start() 

dataset_mount_folder = mount_ctx.mount_point
print(dataset_mount_folder)

files = os.listdir(dataset_mount_folder)
print(files)

# Utworzenie środowiska obliczeniowego

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-gpu-cl-lp"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

# Kopiowanie danych treningowych do skryptu

In [ ]:
import shutil

shutil.copy('./script.py', script_folder)


In [ ]:
with open(os.path.join(script_folder, './script.py'), 'r') as f:
    print(f.read())

# Utworzenie środwiska

In [ ]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.9
- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - tensorflow-gpu
  - tensorflow==2.3.1
  - matplotlib

In [ ]:
from azureml.core import Environment

textgen_env = Environment.from_conda_specification(name = 'texgen', file_path = './conda_dependencies.yml')

# Specify a GPU base image
#textgen_env.docker.enabled = True
#textgen_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'

textgen_env = Environment.get(ws, name='AzureML-TensorFlow-2.3-GPU')

# Konfiguracja treningu

In [ ]:
!pip install tensorflow==2.4.1
import tensorflow as tf
tf.__version__

In [ ]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_named_input('text').as_mount(),
        '--text-id', 10,
        '--text-size', 30000000,
        '--epochs', 2]

src = ScriptRunConfig(source_directory=script_folder,
                      script='script.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=textgen_env)

# Zatwierdzenie zadania do uruchomienia

In [ ]:
run = exp.submit(src)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.get_details()

In [ ]:
run.get_metrics()

In [ ]:
run.get_file_names()

## Pobranie wytrenowanego modelu

In [ ]:
from azureml.core import Experiment, Run

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='textgen')

run = Run(exp, 'textgen_1611390281_8b2cf5c2')

# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/'):
        print(f.replace('outputs/model/', ""))
        output_path = os.path.join('./model/', f.replace('outputs/model/', ""))
        print('Downloading from {} to {} ...'.format(f, output_path))
        run.download_file(name=f, output_file_path=output_path)

In [ ]:
import tensorflow as tf

one_step_reloaded = tf.saved_model.load('model/one_step_model')

## Test

In [ ]:
states = None
next_char = tf.constant(['pan'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8").replace('\r\n', " "))